In [2]:
!git clone https://github.com/YoongiKim/CIFAR-10-images

Cloning into 'CIFAR-10-images'...
remote: Enumerating objects: 60027, done.
remote: Total 60027 (delta 0), reused 0 (delta 0), pack-reused 60027
Receiving objects: 100% (60027/60027), 19.94 MiB | 10.97 MiB/s, done.
Resolving deltas: 100% (59990/59990), done.
Checking out files: 100% (60001/60001), done.


In [3]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torch import nn, optim
from imutils import paths
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler

In [4]:
def if_gpu_avlbl():
    gpu_avlbl = torch.cuda.is_available()
    if not gpu_avlbl:
        print('CUDA is not available.  Training on CPU ...')
    else:
        print('CUDA is available!  Training on GPU ...')

In [5]:
if_gpu_avlbl()

CUDA is available!  Training on GPU ...


In [6]:
#flip, rotation, translation, to_tensor, normalize

train_transform = transforms.Compose([transforms.RandomHorizontalFlip(),
                                      transforms.RandomRotation(10),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), 
                                                          (0.5, 0.5, 0.5))
                                      ])

test_transform = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5),
                                                          (0.5, 0.5, 0.5)),
                                     ])

In [7]:
#defining data loader
test_path = '/content/CIFAR-10-images/test'
test_paths = list(paths.list_images(test_path))
test_data = pd.DataFrame(columns=['image_path', 'label'])
test_labels = []
#classes = []
for i, image_path in enumerate(test_paths):
    test_data.loc[i, 'image_path'] = image_path
    test_label = image_path[len(test_path):].split('/')[1]
    if test_label != 'bird':
        test_label = 'Not_bird'
    #print(test_label)
    test_labels.append(test_label)
    #classes.append((test_label))

for i in range(len(test_labels)):
    test_data.loc[i,"label"] = test_labels[i]

In [8]:
test_data['label'].unique()

array(['Not_bird', 'bird'], dtype=object)

In [9]:
test_data_dum = pd.get_dummies(test_data['label'], drop_first=True)
test_df = pd.concat([test_data['image_path'],test_data_dum], axis=1)
test_df

image_path  bird
0     /content/CIFAR-10-images/test/horse/0115.jpg     0
1     /content/CIFAR-10-images/test/horse/0456.jpg     0
2     /content/CIFAR-10-images/test/horse/0465.jpg     0
3     /content/CIFAR-10-images/test/horse/0946.jpg     0
4     /content/CIFAR-10-images/test/horse/0678.jpg     0
...                                            ...   ...
9995    /content/CIFAR-10-images/test/cat/0246.jpg     0
9996    /content/CIFAR-10-images/test/cat/0268.jpg     0
9997    /content/CIFAR-10-images/test/cat/0940.jpg     0
9998    /content/CIFAR-10-images/test/cat/0896.jpg     0
9999    /content/CIFAR-10-images/test/cat/0444.jpg     0

[10000 rows x 2 columns]

In [10]:
#defining data loader
train_path = '/content/CIFAR-10-images/train'
train_paths = list(paths.list_images(train_path))
train_data = pd.DataFrame(columns=['image_path', 'label'])
train_labels = []
#classes = []
for i, image_path in enumerate(train_paths):
    train_data.loc[i, 'image_path'] = image_path
    train_label = image_path[len(train_path):].split('/')[1]
    if train_label != 'bird':
        train_label = 'Not_bird'
    #print(train_label)
    train_labels.append(train_label)
    #classes.append((train_label))

for i in range(len(train_labels)):
    train_data.loc[i,"label"] = train_labels[i]

In [11]:
train_data['label'].unique()

array(['Not_bird', 'bird'], dtype=object)

In [12]:
train_data_dum = pd.get_dummies(train_data['label'], drop_first=True)
train_df = pd.concat([train_data['image_path'],train_data_dum], axis=1)
train_df

image_path  bird
0      /content/CIFAR-10-images/train/horse/2149.jpg     0
1      /content/CIFAR-10-images/train/horse/1069.jpg     0
2      /content/CIFAR-10-images/train/horse/0115.jpg     0
3      /content/CIFAR-10-images/train/horse/4697.jpg     0
4      /content/CIFAR-10-images/train/horse/2775.jpg     0
...                                              ...   ...
49995    /content/CIFAR-10-images/train/cat/2071.jpg     0
49996    /content/CIFAR-10-images/train/cat/4633.jpg     0
49997    /content/CIFAR-10-images/train/cat/3757.jpg     0
49998    /content/CIFAR-10-images/train/cat/1133.jpg     0
49999    /content/CIFAR-10-images/train/cat/4198.jpg     0

[50000 rows x 2 columns]

In [13]:
train_data_df = train_df.sample(frac=1).reset_index(drop=True) #shuffle the dataset
test_data_df = test_df.sample(frac=1).reset_index(drop=True)

In [14]:
train_data_df

image_path  bird
0          /content/CIFAR-10-images/train/horse/1303.jpg     0
1      /content/CIFAR-10-images/train/automobile/4503...     0
2           /content/CIFAR-10-images/train/deer/3603.jpg     0
3      /content/CIFAR-10-images/train/automobile/0769...     0
4            /content/CIFAR-10-images/train/cat/2014.jpg     0
...                                                  ...   ...
49995   /content/CIFAR-10-images/train/airplane/2314.jpg     0
49996      /content/CIFAR-10-images/train/truck/4244.jpg     0
49997       /content/CIFAR-10-images/train/frog/0243.jpg     0
49998        /content/CIFAR-10-images/train/dog/3912.jpg     0
49999       /content/CIFAR-10-images/train/frog/0925.jpg     0

[50000 rows x 2 columns]

In [15]:
# creating dataset module
class CIFAR2(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.X = self.df.image_path.values
        self.y = self.df.bird.values
        self.transform = transform

    def __len__(self):
        return (len(self.X))

    def __getitem__(self, idx):
        image = Image.open(self.X[idx])
        image = self.transform(image)
        label = self.y[idx]

        return image, label

In [16]:
valid = 0.2
batch_size = 15

train_data = CIFAR2(train_data_df, transform = train_transform)
test_data = CIFAR2(test_data_df, transform = train_transform)

num_train = len(train_data)
indices = list(range(num_train))
split = int(valid * num_train)
train_idx, valid_idx = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
valid_loader = DataLoader(train_data, sampler=valid_sampler, batch_size=batch_size)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle = True)

classes = ['bird', 'Not_bird']

In [17]:
# model = CNN(n_hidden_layers, n_output)

class CNN(nn.Module):
  def __init__(self):
    super(CNN, self).__init__()
    self.conv1 = nn.Conv2d(3,32,3, padding=1)
    self.conv2 = nn.Conv2d(32,64,3, padding=1)
    self.conv3 = nn.Conv2d(64,128,3, padding=1)
    self.conv4 = nn.Conv2d(128,128,3, padding=1)
    self.conv5 = nn.Conv2d(128,256,3, padding=1)
    self.conv6 = nn.Conv2d(256,256,3, padding=1)
    self.pool =  nn.MaxPool2d(2,2)
    self.relu = nn.ReLU(inplace=True)
    self.fc1 = nn.Linear(4096,1024)
    self.fc2 = nn.Linear(1024,512)
    self.fc3 = nn.Linear(512,10)
    self.dropout1 = nn.Dropout(0.2)
    self.dropout2 = nn.Dropout(0.1)
    self.batchnorm1 = nn.BatchNorm2d(32)
    self.batchnorm2 = nn.BatchNorm2d(128)
    self.batchnorm3 = nn.BatchNorm2d(256)

  def forward(self, x):
    x = self.conv1(x)
    x = self.batchnorm1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.relu(x)
    x = self.pool(x)
    
    x = self.conv3(x)
    x = self.batchnorm2(x)
    x = self.relu(x)
    x = self.conv4(x)
    x = self.relu(x)
    x = self.pool(x)
    x = self.dropout1(x)    

    x = self.conv5(x)
    x = self.batchnorm3(x)
    x = self.relu(x)
    x = self.conv6(x)
    x = self.relu(x)
    x = self.pool(x)
    
    x = x.view(x.size(0), -1)
    x = self.dropout1(x)
    x = self.relu(self.fc1(x))
    x = self.dropout2(x)
    x = self.relu(self.fc2(x))
    x = self.fc3(x)
    return x

In [18]:
# Define loss and solver
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.003)

In [19]:
model.cuda()

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv5): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv6): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (relu): ReLU(inplace=True)
  (fc1): Linear(in_features=4096, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=512, bias=True)
  (fc3): Linear(in_features=512, out_features=10, bias=True)
  (dropout1): Dropout(p=0.2, inplace=False)
  (dropout2): Dropout(p=0.1, inplace=False)
  (batchnorm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batchnorm2): BatchNorm2d(128, eps=1e-0

In [20]:
#train_with_validation
# train(n_epoch, model_filename, criterion, optimizer)

def train(epoch, model_name, criterion, optimizer, trainldr, validldr):
  valid_loss_min = np.Inf

  for e in range(epoch):
    train_loss = 0
    valid_loss = 0

    model.train()
    for images, labels in trainldr:
      images, labels = images.cuda(), labels.cuda()
      log_probs = model(images)
      loss = criterion(log_probs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      train_loss += loss.item()*len(images)
    else:
      model.eval()
      with torch.no_grad():
        for images, labels in validldr:
          images, labels = images.cuda(), labels.cuda()
          log_probs = model(images)
          loss = criterion(log_probs, labels)
          valid_loss += loss.item()*len(images)

    train_loss = train_loss/len(trainldr.sampler)
    valid_loss = valid_loss/len(validldr.sampler)

    print("epoch: {}/{}".format(e+1, epoch),
          "train_loss: {:.4f}".format(train_loss),
          "valid_loss: {:.4f}".format(valid_loss))
    
    if valid_loss <= valid_loss_min:
      path = F"/content/{model_name}"
      torch.save(model.state_dict(), path)
      valid_loss_min = valid_loss

In [21]:
epoch = 30
model_name = "cifar2_cnn"
train(epoch, model_name, criterion, optimizer, train_loader, valid_loader)

epoch: 1/30 train_loss: 0.3274 valid_loss: 0.2970
epoch: 2/30 train_loss: 0.2815 valid_loss: 0.2826
epoch: 3/30 train_loss: 0.2634 valid_loss: 0.2734
epoch: 4/30 train_loss: 0.2489 valid_loss: 0.2751
epoch: 5/30 train_loss: 0.2355 valid_loss: 0.2397
epoch: 6/30 train_loss: 0.2222 valid_loss: 0.2214
epoch: 7/30 train_loss: 0.2119 valid_loss: 0.2060
epoch: 8/30 train_loss: 0.2041 valid_loss: 0.1970
epoch: 9/30 train_loss: 0.1980 valid_loss: 0.1957
epoch: 10/30 train_loss: 0.1902 valid_loss: 0.2040
epoch: 11/30 train_loss: 0.1844 valid_loss: 0.1845
epoch: 12/30 train_loss: 0.1794 valid_loss: 0.2038
epoch: 13/30 train_loss: 0.1747 valid_loss: 0.2575
epoch: 14/30 train_loss: 0.1697 valid_loss: 0.1762
epoch: 15/30 train_loss: 0.1642 valid_loss: 0.1805
epoch: 16/30 train_loss: 0.1605 valid_loss: 0.1669
epoch: 17/30 train_loss: 0.1569 valid_loss: 0.1659
epoch: 18/30 train_loss: 0.1517 valid_loss: 0.1704
epoch: 19/30 train_loss: 0.1492 valid_loss: 0.1657
epoch: 20/30 train_loss: 0.1426 valid_lo

In [22]:
# Evaluation with inference: load model
# performance = test(model_filename) # total accuracy

def test(model_name):
    path = F"/content/{model_name}"
    model.load_state_dict(torch.load(path))

    test_loss = 0
    test_accuracy = 0
    
    model.eval()
    for images, labels in test_loader:
        images, labels = images.cuda(), labels.cuda()
        output = model(images)
        loss = criterion(output, labels)
        test_loss += loss.item()
        probs = torch.exp(output)
        top_prob, top_class = probs.topk(1, dim=1)
        equals = top_class == labels.view(top_class.shape)
        test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()
    
    test_loss = test_loss/len(test_loader)
    test_accuracy = test_accuracy/len(test_loader)
    print('Test Loss: {:.3f}\n'.format(test_loss),
          'Test Accuracy: {:.3f}'.format(test_accuracy))
    fp = open("/content/performance.txt", "x")
    fp.write('Test Loss: {:.3f}... Test Accuracy: {:.3f}'.format(test_loss, test_accuracy))
    fp.close()

In [23]:
test('cifar2_cnn')

Test Loss: 0.148
 Test Accuracy: 0.948
